Importando Bibliotecas

In [49]:
from math import remainder

from keras.layers import Dense, Input, Activation, Dropout
from keras.models import Model

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

import numpy as np
import os
import time
import pandas as pd


Importando dataset

In [31]:
from settings import PATH, DATABASE_PATH
csv = "Video_Game_Sales.csv"
df = pd.read_csv(os.path.join(DATABASE_PATH, csv))
df.columns = df.columns.str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.54,76.0,51.0,8.0,324.0,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.80,3.79,3.29,35.57,82.0,73.0,8.3,712.0,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.95,3.28,2.95,32.78,80.0,73.0,8.0,193.0,E
4,Pokemon Red/Pokemon Blue,G,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN


In [32]:
df.isnull().sum()


Name                  0
Platform              0
Year_of_Release       8
Genre                 0
Publisher             1
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
Critic_Score       9080
Critic_Count       9080
User_Score         9618
User_Count         9618
Rating             7164
dtype: int64

In [37]:
df.drop(['Global_Sales', 'Other_Sales'], axis=1, inplace=True)
df.dropna(axis=0, inplace=True)
df.head()


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,76.0,51.0,8.0,324.0,E
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.80,3.79,82.0,73.0,8.3,712.0,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.95,3.28,80.0,73.0,8.0,193.0,E
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.15,6.50,89.0,65.0,8.5,433.0,E
7,Wii Play,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,58.0,41.0,6.6,129.0,E


In [38]:
dados = df.loc[df['NA_Sales'] > 1]
dados = df.loc[df['EU_Sales'] > 1]
dados.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,76.0,51.0,8.0,324.0,E
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.80,3.79,82.0,73.0,8.3,712.0,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.95,3.28,80.0,73.0,8.0,193.0,E
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.15,6.50,89.0,65.0,8.5,433.0,E
7,Wii Play,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,58.0,41.0,6.6,129.0,E


In [39]:
name = dados.Name
dados.drop('Name', axis=1, inplace=True)

C:\Users\mugiw\AppData\Local\Temp\ipykernel_5736\1774814663.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados.drop('Name', axis=1, inplace=True)


In [42]:
name

0                                  Wii Sports
2                              Mario Kart Wii
3                           Wii Sports Resort
6                       New Super Mario Bros.
7                                    Wii Play
                        ...                  
1350    Classic NES Series: Super Mario Bros.
1373           Dragon Quest Monsters: Joker 2
1448                         Samurai Warriors
1466                   Monster Hunter Freedom
1587       Tamagotchi Connection: Corner Shop
Name: Name, Length: 88, dtype: object

In [40]:
dados.head()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,76.0,51.0,8.0,324.0,E
2,Wii,2008.0,Racing,Nintendo,15.68,12.80,3.79,82.0,73.0,8.3,712.0,E
3,Wii,2009.0,Sports,Nintendo,15.61,10.95,3.28,80.0,73.0,8.0,193.0,E
6,DS,2006.0,Platform,Nintendo,11.28,9.15,6.50,89.0,65.0,8.5,433.0,E
7,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,58.0,41.0,6.6,129.0,E


In [23]:
dados.shape

(334, 5)

Acessando as colunas da base de dados, separando-as em vendas na america do norte, europa e japão.

In [53]:
venda_na = dados.iloc[:,4].values
venda_jp = dados.iloc[:,6].values
venda_eu = dados.iloc[:,5].values


Criando uma variável chamada previsores, que armazena uma seleção específica de colunas do dataset dados.
sendo elas:
* 0: Name (Nome do jogo)
* 1: Platform (Plataforma do jogo)
* 2: Year_of_Release (Ano de lançamento)
* 3: Genre (Gênero do jogo)
* 7: Publisher (Editor do jogo)
* 8: Critic_Score (Pontuação dos críticos)
* 9: Critic_Count (Número de críticas)
* 10: User_Score (Pontuação dos usuários)
* 11: User_Count (Número de análises dos usuários)

In [54]:
previsores = dados.iloc[:,[0,1,2,3,7,8,9,10,11]].values

Aplicando o _ColumnTransformer_ para transformar variáveis categóricas em variáveis numéricas, utilizando *OneHotEncoder* para codificação, e mantendo as outras variáveis como estão.

- O ColumnTransformer é utilizado para aplicar transformações específicas em diferentes colunas de um conjunto de dados. Ele permite combinar várias operações em um único pipeline de transformação.
- OneHotEncoder cria uma variável binária (0 ou 1) para cada categoria distinta nessas colunas. Por exemplo, para a coluna Genre (Gênero do jogo), será criada uma coluna para cada gênero (Ação, Aventura, etc.), e o valor será 1 ou 0, dependendo se o jogo pertence a esse gênero ou não, enquanto as demais colunas são passadas sem alteração, por conta do parametro 'passThrough'

In [55]:
ct = ColumnTransformer([('columns', OneHotEncoder(), [0,2,3,8])], remainder = 'passthrough')
previsores = ct.fit_transform(previsores).toarray()

In [59]:
input_size = previsores.shape

(88, 42)

In [58]:
(previsores.shape[1] + 3) / 2

22.5

Criando o modelo:
- Uma camada de entrada de entrada com o shape igual o tamanho dos _previsores_.
- Duas camadas ocultas, cada uma com 32 neurônios e ativação sigmoide.
- Três camadas de saída, cada uma com 1 neurônio e ativação linear, para prever as vendas das 3 regiões diferentes

Para o treinamento usamos os previsores, assim como as vendas das 3 regiões para um batch_size de 100, e 5000 epocas.

In [63]:
input_layer = Input(shape=(input_size,))
hidden_layer = Dense(units=32, activation='sigmoid')(input_layer)
hidden_layer_2 = Dense(units=32, activation='sigmoid')(hidden_layer)
output_layer = Dense(units=1, activation='linear')(hidden_layer_2)
output_layer_2 = Dense(units=1, activation='linear')(hidden_layer_2)
output_layer_3 = Dense(units=1, activation='linear')(hidden_layer_2)

regression = Model(inputs=input_layer, outputs=[output_layer, output_layer_2, output_layer_3])
regression.compile(optimizer='adam', loss='mse')
regression.fit(previsores, [venda_na, venda_eu, venda_jp], batch_size=100, epochs=5000)

Epoch 1/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - dense_17_loss: 34.9745 - dense_18_loss: 18.4461 - dense_19_loss: 15.0101 - loss: 68.4307
Epoch 2/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - dense_17_loss: 34.8494 - dense_18_loss: 18.3632 - dense_19_loss: 14.6477 - loss: 67.8604
Epoch 3/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - dense_17_loss: 34.7257 - dense_18_loss: 18.2804 - dense_19_loss: 14.2928 - loss: 67.2990
Epoch 4/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - dense_17_loss: 34.6021 - dense_18_loss: 18.1982 - dense_19_loss: 13.9397 - loss: 66.7400
Epoch 5/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - dense_17_loss: 34.4799 - dense_18_loss: 18.1184 - dense_19_loss: 13.5952 - loss: 66.1936
Epoch 6/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - dense_17_loss: 34.3585 - dense_18_loss: 18.0407 - dense_19_loss: 13.2587 - loss: 65.6580
Epoch 7/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - dense_17_loss: 34.2379 - dense_18_loss: 17.9636 - dense_19_loss: 12.9283 - loss: 65.1298
Epoch

In [64]:
previsao_eu, previsao_jp, previsao_na = regression.predict(previsores)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step


In [65]:
print("Média previsão EU:", previsao_eu.mean(), "Média valores reais EU:", venda_eu.mean())

Média previsão EU: 3.2958117 Média valores reais EU: 2.47


In [66]:
print("Média previsão JP:", previsao_jp.mean(), "Média valores reais JP:", venda_jp.mean())

Média previsão JP: 2.4683657 Média valores reais JP: 2.0584090909090906


In [67]:
print("Média previsão NA:", previsao_na.mean(), "Média valores reais NA:", venda_na.mean())

Média previsão NA: 2.055781 Média valores reais NA: 3.297386363636363
